# INM702 Coursework Task 1
### Aaron Mir (Student Number:160001207)


<img src="Task1(1).jpg" alt="Task1(1)" width="700"/>
<img src="Task1(2).jpg" alt="Task1(2)" width="700"/>
<img src="Task1(3).jpg" alt="Task1(3)" width="700"/>
<img src="Task1(4).jpg" alt="Task1(4)" width="700"/>


The  first  task  tests  your  Python  skills.  You  need  to  develop  a  simple  game  consisting  of  a rectangular grid (of size height x width ) where each cell has a random integer value between 0  and 9.An  agent  starts  at  the  upper-left  cornerof  the  gridand  must  reach  the  lower-right corner of the grid as fast as possible.You can implement one of the two (or both, for no extra point) game modes:

    • The time spent on a cell is the number on this cell
    • The time spent on a cell is the absolute of the difference between the previous cell the agent was on and the current cell it is on

In order to solve this problem, you will provide 3 algorithms:

    • A first baseline of your choosing. E.g. it can be any search algorithm, or an algorithm using heuristics. It doesn’thave  to  perform  fastor  well,  but  should be  better  than random movements.
    • Dijkstra'salgorithm
    • Antcolony optimization algorithm

You should describe the algorithms and compare them. Are they always solving the problem? How long do they take depending on the size of the maze?


In [ ]:
from __future__ import division
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
import sys
import random
import copy

# Use assert to test functions
# Use a class to create the environment
# Do the task 2 first

In [ ]:
class Agent:
    def __init__ (self, name):
        self.name = name

    def moveUp(self):
    
    def moveDown(self):
    
    def moveLeft(self):
    
    def moveRight(self):

class IntMaze:
    def __init__ (self, agent, rows, columns):
        self.rows = rows
        self.columns = columns
    